## 11.1 Off the shelf results with T5

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [2]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-base')
base_tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the 

## Abstractive Summarization

In [3]:
text_to_summarize ="""Sinan Ozdemir is a data scientist, startup founder, and educator living in the San Francisco Bay Area with his dog, 
Charlie; cat, Euclid; and bearded dragon, Fiero. He spent his academic career studying pure mathematics 
at Johns Hopkins University before transitioning to education. He spent several years conducting lectures 
on data science at Johns Hopkins University and at the General Assembly before founding his own startup, 
Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts. 
After completing a Fellowship at the Y Combinator accelerator, Sinan spent most of his time working on 
his fast-growing company, while creating educational material for data science.
"""

preprocess_text = text_to_summarize.strip().replace("\n","")

print ("original text preprocessed:\n", preprocess_text)

original text preprocessed:
 Sinan Ozdemir is a data scientist, startup founder, and educator living in the San Francisco Bay Area with his dog, Charlie; cat, Euclid; and bearded dragon, Fiero. He spent his academic career studying pure mathematics at Johns Hopkins University before transitioning to education. He spent several years conducting lectures on data science at Johns Hopkins University and at the General Assembly before founding his own startup, Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts. After completing a Fellowship at the Y Combinator accelerator, Sinan spent most of his time working on his fast-growing company, while creating educational material for data science.


In [4]:
# known prompt for summarization with T5
t5_prepared_text = "summarize: " + preprocess_text

input_ids = base_tokenizer.encode(t5_prepared_text, return_tensors="pt")

# summmarize 
summary_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    min_length=30,
    max_length=50,
    early_stopping=True
)

output = base_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print (f"Summarized text: \n{output}")

Summarized text: 
Sinan Ozdemir is a data scientist, startup founder, and educator. he founded his own startup, Kylie.ai, which uses artificial intelligence to build chatbots.


## English -> German Translation

In [5]:
input_ids = base_tokenizer.encode('translate English to German: Where is the chocolate?', return_tensors='pt')

# translate 
translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print (f"Translated text:\n{output}")


Translated text:
Wo ist die Schokolade?


In [6]:
# pass labels in to calculate loss

input_ids = base_tokenizer('translate English to German: Where is the chocolate?', return_tensors='pt').input_ids
labels = base_tokenizer('Wo ist die Schokolade?', return_tensors='pt').input_ids

loss = base_model(input_ids=input_ids, labels=labels).loss

labels, loss

(tensor([[ 3488,   229,    67, 31267,    58,     1]]),
 tensor(0.1136, grad_fn=<NllLossBackward0>))

## CoLA: The Corpus of Linguistic Acceptability
checking for grammatical correctess

In [7]:
input_ids = base_tokenizer.encode('cola sentence: Where is the chocolate?', return_tensors='pt')

# CoLA 
translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")


is grammatically correct?: 
acceptable


In [8]:
input_ids = base_tokenizer.encode('cola sentence: Where be a chocolates?', return_tensors='pt')

# summmarize 
translate_ids = base_model.generate(
    input_ids,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")

is grammatically correct?: 
unacceptable


/usr/local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


## STSB - Semantic Text Similarity Benchmark
Are two sentences semantically similar

In [9]:
sentence_one = 'How to fish'
sentence_two = 'Fishing Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# calculate semantic similarity 
translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

semantically similar? (0-5): 
3.2


In [10]:
sentence_one = 'How to fish'
sentence_two = 'Hiking Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# calculate semantic similarity
translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

semantically similar? (0-5): 
0.4


## MNLI - Multi-Genre Natural Language Inference
Whether a premise implies (“entailment”), contradicts (“contradiction”), or neither (“neutral”) a hypothesis.

In [11]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I am running for mayor', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
entailment


/usr/local/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [12]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I do not really vote', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
contradiction


In [13]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I code for a living', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
neutral


## Q/A - Question/Answering

In [14]:
input_ids = base_tokenizer.encode(
    'question: Where does Sinan live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
California


In [15]:
input_ids = base_tokenizer.encode(
    'question: Where does Matt live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
Boston


In [16]:
# Sanity check with random prompts

input_ids = base_tokenizer.encode(
    'prompt1: Where does Matt live? prompt2: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
prompt2


## 11.2 Using T5 for abstractive summarization

In [17]:
from transformers import pipeline, T5ForConditionalGeneration, TrainingArguments, Trainer, \
                         DataCollatorForSeq2Seq
import pandas as pd
from datasets import Dataset
import random

In [18]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-small')
base_tokenizer = T5Tokenizer.from_pretrained('t5-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
# https://www.kaggle.com/snap/amazon-fine-food-reviews?select=Reviews.csv

reviews = pd.read_csv('data/reviews.csv')

# Pre-processing step
# Punctuation is important in grammar and important for complex decoding architectures to know when to stop!
def add_punc(s):
    if s[-1] not in ('.', '!', '?'):
        s = s + '.'
    return s

reviews.dropna(inplace=True)

reviews['Summary'] = reviews['Summary'].map(add_punc)

print(reviews.shape)

reviews.head()

(96486, 3)


,Text,Summary,Score
0,Great taffy at a great price. There was a wid...,Great taffy.,5
1,This taffy is so good. It is very soft and ch...,"Wonderful, tasty taffy.",5
2,Right now I'm mostly just sprouting this so my...,Yay Barley.,5
3,This is a very healthy dog food. Good for thei...,Healthy Dog Food.,5
4,good flavor! these came securely packed... the...,fresh and greasy!,4


In [20]:
reviews = reviews[(reviews['Summary'].str.len() < 100) & (reviews['Summary'].str.len() >=30)]

reviews.shape

(13073, 3)

In [21]:
random.seed(0)

reviews_dataset = Dataset.from_pandas(reviews.astype(str).sample(50))

In [22]:
# We have a prompt but only as a prefix in the encoder
prefix = "summarize: "

# we will manually add our own labels because unlike GPT, we cannot assume the labels are based on the inputs
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Text"]]
    model_inputs = base_tokenizer(inputs, max_length=1024, truncation=True)

    labels = base_tokenizer(examples["Summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [23]:
tokenized_reviews_dataset = reviews_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [24]:
tokenized_reviews_dataset = tokenized_reviews_dataset.train_test_split(test_size=.1)

In [25]:
# Data collator specifically for generic sequence to sequence tasks
# Use when we are translating one sequence to another like translation, summarization, etc
data_collator = DataCollatorForSeq2Seq(tokenizer=base_tokenizer, model=base_model)

In [26]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [27]:
training_args = TrainingArguments(
    output_dir="./t5_summary_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    load_best_model_at_end=True,
    logging_steps=50,
    save_strategy='epoch'
)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_reviews_dataset["train"],
    eval_dataset=tokenized_reviews_dataset["test"],
    data_collator=data_collator,
)

trainer.evaluate()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


{'eval_loss': 3.656369686126709,
 'eval_runtime': 0.9231,
 'eval_samples_per_second': 5.417,
 'eval_steps_per_second': 1.083}

In [28]:
trainer.train()  # total of 9 hours of training on my laptop!

Epoch,Training Loss,Validation Loss
1,No log,3.490143
2,No log,3.259648
3,No log,3.162967
4,No log,3.089562
5,No log,3.031952
6,No log,2.981776
7,No log,2.931195
8,No log,2.907024
9,No log,2.885082
10,No log,2.867713


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=60, training_loss=2.4075971603393556, metrics={'train_runtime': 271.6358, 'train_samples_per_second': 3.313, 'train_steps_per_second': 0.221, 'total_flos': 11245264797696.0, 'train_loss': 2.4075971603393556, 'epoch': 20.0})

In [29]:
trainer.evaluate()

{'eval_loss': 2.8142893314361572,
 'eval_runtime': 0.3047,
 'eval_samples_per_second': 16.408,
 'eval_steps_per_second': 3.282,
 'epoch': 20.0}

In [30]:
trainer.save_model()

In [31]:
loaded_model = T5ForConditionalGeneration.from_pretrained('./t5_summary_results')

# summarization pipeline prepends a default prefix of summarize: 
generator = pipeline(
    'summarization', model=loaded_model, tokenizer=base_tokenizer
)

In [32]:
sam = reviews.sample(1)

print(sam['Summary'])

text = sam['Text'].tolist()[0]
text

74105    Great Coffee at a great value.
Name: Summary, dtype: object


'I am a coffee fanatic.  This coffee is delicious!  As good if not better than other brands, and the price is reasonable.'

In [33]:
# Generate a summary
generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'I am a coffee fanatic.'}]

In [34]:
# Try the base t5 on the same text
base_generator = pipeline(
    'summarization', model='t5-small', tokenizer='t5-small'
)

# Summary is a bit more extractive than our fine-tuned version and style isn't quite the same as our dataset
base_generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'a coffee fanatic, this coffee is delicious!'}]

In [35]:
# Sanity check: trying a different prefix. Not a good result

inputs = base_tokenizer("not my prompt: " + text, return_tensors="pt")
outputs = loaded_model.generate(
    inputs["input_ids"], min_length=3, max_length=15
)

print(base_tokenizer.decode(outputs[0], skip_special_tokens=True))

prompt: I am a coffee fanatic.
